<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/AccuracyFeedback0530.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install gradio speechrecognition python-Levenshtein soundfile

# 1. Score only

This app is uploaded as pronunciationfeedback (0530)

# This app is loaded

In [ ]:
#@markdown Output: Accuracy Score

import gradio as gr
import speech_recognition as sr
from Levenshtein import ratio
import tempfile
import numpy as np
import soundfile as sf
import pandas as pd

# Sample dataframe with sentences ordered from easy to hard
data = {
    "Sentences": [
        "A stitch in time saves nine.",
        "To be or not to be, that is the question.",
        "Five cats were living in safe caves.",
        "Hives give shelter to bees in large caves.",
        "His decision to plant a rose was amazing.",
        "She sells sea shells by the sea shore.",
        "The colorful parrot likes rolling berries.",
        "Time flies like an arrow; fruit flies like a banana.",
        "Good things come to those who wait.",
        "All human beings are born free and equal in dignity and rights."
    ]
}
df = pd.DataFrame(data)

def transcribe_audio(file_info):
    r = sr.Recognizer()
    with tempfile.NamedTemporaryFile(delete=True, suffix=".wav") as tmpfile:
        sf.write(tmpfile.name, data=file_info[1], samplerate=44100, format='WAV')
        tmpfile.seek(0)
        with sr.AudioFile(tmpfile.name) as source:
            audio_data = r.record(source)
    try:
        text = r.recognize_google(audio_data)
        return text
    except sr.UnknownValueError:
        return "Could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results; {e}"

def pronunciation_correction(expected_text, file_info):
    user_spoken_text = transcribe_audio(file_info)
    similarity = ratio(expected_text.lower(), user_spoken_text.lower())
    description = f"{similarity:.2f}"

    if similarity >= 0.9:
        feedback = "Excellent pronunciation!"
    elif similarity >= 0.7:
        feedback = "Good pronunciation!"
    elif similarity >= 0.5:
        feedback = "Needs improvement."
    else:
        feedback = "Poor pronunciation, try to focus more on clarity."

    return feedback, description

with gr.Blocks() as app:
    with gr.Row():
        sentence_dropdown = gr.Dropdown(choices=df['Sentences'].tolist(), label="Select a Sentence")
        selected_sentence_output = gr.Textbox(label="Selected Text", interactive=False)
    audio_input = gr.Audio(label="Upload Audio File", type="numpy")
    check_pronunciation_button = gr.Button("Check Pronunciation")
    pronunciation_feedback = gr.Textbox(label="Pronunciation Feedback")
    pronunciation_score = gr.Number(label="Pronunciation Accuracy Score: 0 (No Match) ~ 1 (Perfect)")

    sentence_dropdown.change(lambda x: x, inputs=sentence_dropdown, outputs=selected_sentence_output)
    check_pronunciation_button.click(
        pronunciation_correction,
        inputs=[sentence_dropdown, audio_input],
        outputs=[pronunciation_feedback, pronunciation_score]
    )

app.launch(debug=True)


# User average

# This works giving the final average for the given name

In [ ]:
#@markdown Accuracy Score, Average, user name
import gradio as gr
import speech_recognition as sr
from Levenshtein import ratio
import tempfile
import numpy as np
import soundfile as sf
import pandas as pd

# Sample dataframe with sentences
data = {
    "Sentences": [
        "A stitch in time saves nine.",
        "To be or not to be, that is the question.",
        "Five cats were living in safe caves.",
        "Hives give shelter to bees in large caves.",
        "His decision to plant a rose was amazing.",
        "She sells sea shells by the sea shore.",
        "The colorful parrot likes rolling berries.",
        "Time flies like an arrow; fruit flies like a banana.",
        "Good things come to those who wait.",
        "All human beings are born free and equal in dignity and rights."
    ]
}
df = pd.DataFrame(data)
user_scores = {}

def transcribe_audio(file_info):
    r = sr.Recognizer()
    with tempfile.NamedTemporaryFile(delete=True, suffix=".wav") as tmpfile:
        sf.write(tmpfile.name, data=file_info[1], samplerate=44100, format='WAV')
        tmpfile.seek(0)
        with sr.AudioFile(tmpfile.name) as source:
            audio_data = r.record(source)
    try:
        text = r.recognize_google(audio_data)
        return text
    except sr.UnknownValueError:
        return "Could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results; {e}"

def pronunciation_correction(name, expected_text, file_info):
    user_spoken_text = transcribe_audio(file_info)
    similarity = ratio(expected_text.lower(), user_spoken_text.lower())
    score = float(f"{similarity:.2f}")
    if name in user_scores:
        user_scores[name].append(score)  # Track scores for each user
    else:
        user_scores[name] = [score]
    feedback = "Excellent pronunciation!" if score >= 0.9 else \
               "Good pronunciation!" if score >= 0.7 else \
               "Needs improvement." if score >= 0.5 else \
               "Poor pronunciation, try to focus more on clarity."
    return feedback, score

def calculate_average(name):
    if name in user_scores and user_scores[name]:
        filtered_scores = [score for score in user_scores[name] if score > 0]  # Ignore zeros
        average_score = sum(filtered_scores) / len(filtered_scores)
    else:
        average_score = 0
    return f"😍 Great job, {name}! \n\nYour average score (excluding zeros) is: {average_score:.2f}. \nRemember, this score only focuses on the accuracy of individual sounds. \nKeep up the fun and enjoyment as you continue learning English!"

with gr.Blocks() as app:
    name_input = gr.Textbox(label="Enter your name", placeholder="Type your name here...", value="")
    with gr.Row():
        sentence_dropdown = gr.Dropdown(choices=df['Sentences'].tolist(), label="Select a Sentence")
        selected_sentence_output = gr.Textbox(label="Selected Text", interactive=False)
    audio_input = gr.Audio(label="Upload Audio File", type="numpy")
    check_pronunciation_button = gr.Button("Check Pronunciation")
    pronunciation_feedback = gr.Textbox(label="Pronunciation Feedback")
    pronunciation_score = gr.Number(label="Pronunciation Accuracy Score: 0 (No Match) ~ 1 (Perfect)")
    complete_button = gr.Button("Complete")
    average_output = gr.Textbox(label="Average Score Output", visible=True)

    sentence_dropdown.change(lambda x: x, inputs=sentence_dropdown, outputs=selected_sentence_output)
    check_pronunciation_button.click(
        pronunciation_correction,
        inputs=[name_input, sentence_dropdown, audio_input],
        outputs=[pronunciation_feedback, pronunciation_score]
    )
    complete_button.click(
        calculate_average,
        inputs=[name_input],
        outputs=average_output
    )

app.launch(debug=True)
